In [1]:
# Run these following 3 installs to if you don't have shapely, geopands, etc
# if you are using sagemaker, you should use the "conda_python3" kernel
!pip install laspy
!pip install geopandas
# !pip install shapely

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
from laspy.file import File
from pandas import DataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point
import requests

In [3]:
#import some GIS visualization libraries - ipyleaflet is not working in sagemaker.  Will use folium instead
# !pip install ipyleaflet


In [4]:
#install folium mapping tools
!pip install folium

    100% |████████████████████████████████| 92kB 14.5MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
# url = "https://dc-lidar-2018.s3.amazonaws.com/Classified_LAS/1120.las"
# lidar_data = requests.get(url)
# with open("./1120.las", 'wb') as f:
#     f.write(lidar_data.content)

Load a set of example lidar points (grid 1120) from our repo
The entire set of DC grids are here: 
https://dcgis.maps.arcgis.com/apps/View/index.html?appid=f0145164d65848dc978e887063a53f25

In [6]:
inFile = File("./1120.las")

Load in as much as of inFile fields as we can.  We can decide what to do with it later. 

In [7]:
lidar_points = np.array((inFile.X,inFile.Y,inFile.Z,inFile.intensity,
                          inFile.classification, inFile.gps_time, 
                          inFile.overlap, inFile.scan_angle, 
                          inFile.x, inFile.y, inFile.z, 
                          inFile.Synthetic, inFile.Withheld)).transpose()

In [8]:
lidar_df=DataFrame(lidar_points, columns = ["X","Y","Z","intensity",
                                            "classification","gps_time",
                                            "overlap","scan_angle", 
                                            "x", "y", "z",
                                            "Synthentic", "Withheld"])

Take a look at how many points there are present: 

In [9]:
lidar_df.shape

(3588649, 13)

## Converstion of LAS's coordinate system to Lat Long (ESPG 4326) standard coordinate system

LIDAR looks to be using a coordinate system specified in its metadata file: https://dc-lidar-2015.s3.amazonaws.com/Classified_LAS/DC_Octo_Classified_LAS.xml
Looking into the last 2 lines of the LDR Info block, we have 2 some leads: it looks like the coordinate system is some kind of NAVD88

```
<ldrinfo>
<ldrspec>USGS v1.2 QL1</ldrspec>
<ldrsens>Leica ALS70</ldrsens>
<ldrmaxnr>7</ldrmaxnr>
<ldrnps>0.26</ldrnps>
<ldrdens>13.3</ldrdens>
<ldradens>14</ldradens>
<ldranps>0.25</ldranps>
<ldrfltht>1045</ldrfltht>
<ldrfltsp>140</ldrfltsp>
<ldrscana>25</ldrscana>
<ldrscanr>66.3</ldrscanr>
<ldrpulsr>500</ldrpulsr>
<ldrpulsd>10</ldrpulsd>
<ldrpulsw>3</ldrpulsw>
<ldrwavel>1064</ldrwavel>
<ldrmpia>1</ldrmpia>
<ldrbmdiv>0.3</ldrbmdiv>
<ldrswatw>1650</ldrswatw>
<ldrswato>500</ldrswato>
<ldrcrs>NAD_1983_StatePlane_Maryland_FIPS_1900 (Meters)</ldrcrs>
<ldrgeoid>NAVD88 - Geoid12A (Meters)</ldrgeoid>
</ldrinfo>
```

Use the search feature of EPSG.IO coordinate conversion site and looking for NAD83, I found a coordinate system that might match the this LAS data: NAD83 EPSG 26985
https://epsg.io/transform

Let's take a look at the Lidar data first: 

In [10]:
lidar_df.head()

,X,Y,Z,intensity,classification,gps_time,overlap,scan_angle,x,y,z,Synthentic,Withheld
0,38940126.0,14019987.0,5512.0,16750.0,2.0,2.070124e+08,1.0,7.0,389401.26,140199.87,55.12,0.0,0.0
1,38940062.0,14019945.0,5518.0,8442.0,2.0,2.070124e+08,1.0,7.0,389400.62,140199.45,55.18,0.0,0.0
2,38940043.0,14019872.0,5526.0,9112.0,2.0,2.070124e+08,1.0,7.0,389400.43,140198.72,55.26,0.0,0.0
3,38940117.0,14019920.0,5522.0,10854.0,2.0,2.070124e+08,1.0,7.0,389401.17,140199.20,55.22,0.0,0.0
4,38940189.0,14019967.0,5516.0,20502.0,2.0,2.070124e+08,1.0,7.0,389401.89,140199.67,55.16,0.0,0.0


Conversion of LAS's coordinate to lat long should reasonably fit into the bound of DC, this will serve as a first cut confirmation of horizontal datum conversion process

The LIDAR readme file: https://docs.opendata.aws/dc-lidar-2018/readme.html contains the following bounding box.  
```
<spdom>
<bounding>
<westbc>-77.122373</westbc>
<eastbc> -76.900716</eastbc>
<northbc>39.001746</northbc>
<southbc>38.785481</southbc>
</bounding>
</spdom>
```

Again, using the espg.io site's tranform function, I did a quick conversion.  The result looks promising
![title](img/espg-conversion-one-sample-point.png)

In [11]:
import random as rnd
from random import uniform

In [12]:
#Use df.sample to randomly sample points
# num_sampled_points = 5000
# lidar_df_sampled = 

In [13]:
num_sampled_points = 5
lidar_df_sampled = lidar_df.head(num_sampled_points)

In [14]:
geometry = [Point(xyz) for xyz in zip(inFile.x,inFile.y,inFile.z)]

In [15]:
geometry_sampled = geometry[0:num_sampled_points]

In [16]:
crs = None
lidar_geodf_sampled = GeoDataFrame(lidar_df_sampled, crs=crs, geometry=geometry_sampled)
lidar_geodf_sampled.crs = {'init': 'epsg:26985'}
lidar_geodf_sampled['geometry'] = lidar_geodf_sampled['geometry'].to_crs(epsg=4326)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [17]:
lidar_geodf_sampled['geometry'].head()

0    POINT Z (-77.12223 38.92961 55.12000)
1    POINT Z (-77.12224 38.92961 55.18000)
2    POINT Z (-77.12224 38.92960 55.26000)
3    POINT Z (-77.12223 38.92961 55.22000)
4    POINT Z (-77.12223 38.92961 55.16000)
Name: geometry, dtype: geometry

In [18]:
# import geopandas as gpd
# import matplotlib.pyplot as plt

In [19]:
import folium

In [20]:
#here's an exmaple to access lidar_geo's coordinates: x, and y
[lidar_geodf_sampled['geometry'][0].x, lidar_geodf_sampled['geometry'][0].y, lidar_geodf_sampled['geometry'][0].z]

[-77.12223228002982, 38.92961481462707, 55.120000000000005]

In [21]:
#let's display a blank DC map
DC_Coord = [38.9072, -77.0369]

#build map
map_dc = folium.Map(location=DC_Coord, zoom_start=13, tiles='cartodbpositron', width=640, height=480)
map_dc

In [22]:
folium_sample_coords = [[lidar_geodf_sampled['geometry'][i].y, lidar_geodf_sampled['geometry'][i].x] 
                        for i in range(num_sampled_points)]

In [23]:
#build some folium markers to mark our LAS points
#folium points are (lat, long)
folium_sample_coords

[[38.92961481462707, -77.12223228002982],
 [38.92961102341192, -77.12223965465381],
 [38.92960444504836, -77.12224183464343],
 [38.929608777969214, -77.1222333076484],
 [38.929613020564986, -77.12222501115762]]

In [24]:

[folium.CircleMarker(folium_sample_coords[i], radius=1,
                color='#0080bb', fill_color='#0080bb').add_to(map_dc) 
for i in range(num_sampled_points)]

In [25]:
#plot out the sampled points for Octo grid, 1120, which is within North Arlington
#and... success!
map_dc